### Sentiment Engine training

#### proper installation and import and mount on google drive

In [2]:
!pip install transformers
from transformers import RobertaForSequenceClassification, RobertaTokenizer, BertForSequenceClassification, BertTokenizer, AutoModelForSequenceClassification, AutoTokenizer, AdamW

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
cd drive/MyDrive/Projects/PLP_PracticeModule/

/content/drive/MyDrive/Projects/PLP_PracticeModule


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
%matplotlib inline

#### read dataset and replace text labels to binary labels

In [27]:
df = pd.read_csv('final_dataset_for_training.csv',usecols=['body','entities','Stock'])
df

,body,entities,Stock
0,$TSLA stock starting to rocket 🚀,Bullish,TSLA
1,$TSLA Money got banana it grow big \nBut took ...,Bullish,TSLA
2,"$AAPL bear logic last qtr, but the iPhone numb...",Bullish,AAPL
3,$AAPL 129 at the morning bell. Sleep well my B...,Bullish,AAPL
4,$AMZN $TSLA $AAPL $GOOGL now well get some che...,Bullish,TSLA
...,...,...,...
3199995,$TSLA short the rip until we hit $900,Bearish,TSLA
3199996,$AAPL,Bearish,AAPL
3199997,$AMZN think its going to fill 3670 first...? T...,Bearish,AMZN
3199998,@LuckyLloydChristmas @Bullytrading Im such a f...,Bearish,TSLA


In [28]:
df['entities'] = df.entities.replace('Bullish',1).replace('Bearish',0)

#### preprocessing

In [29]:
!pip install emoji
import emoji
import re

def process_text(texts):
  # lowercase
  # message = message.lower() # RoBERTa tokenizer is uncased
  # remove URLs
  texts = re.sub(r'https?://\S+', "", texts)
  texts = re.sub(r'www.\S+', "", texts)
  # remove '
  texts = texts.replace('&#39;', "'")
  # remove symbol names
  texts = re.sub(r'(\#)(\S+)', r'hashtag_\2', texts)
  texts = re.sub(r'(\$)([A-Za-z]+)', r'cashtag_\2', texts)
  # remove usernames
  texts = re.sub(r'(\@)(\S+)', r'mention_\2', texts)
  # demojize
  texts = emoji.demojize(texts, delimiters=("", " "))


  return texts.strip()
    

     |████████████████████████████████| 175 kB 4.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=83abff243d6a9a74a49d5b48108f9dc0c97c0d1f8e3298a669ca33bed1fffb7d
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [30]:
df['body'] = df.body.apply(process_text)
df.head()

,body,entities,Stock
0,cashtag_TSLA stock starting to rocket rocket,1,TSLA
1,cashtag_TSLA Money got banana it grow big \nBu...,1,TSLA
2,"cashtag_AAPL bear logic last qtr, but the iPho...",1,AAPL
3,cashtag_AAPL 129 at the morning bell. Sleep we...,1,AAPL
4,cashtag_AMZN cashtag_TSLA cashtag_AAPL cashtag...,1,TSLA


In [31]:
labels = df.entities.values
text = df.body.values

#### check the tokens number distribution of all the text data entries

In [9]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
def helper(text):
  t = word_tokenize(text)
  return len(t)
length = df.body.apply(helper)

print('there are {} entries in the dataframe'.format(len(length[length>=0])))
print('{}% of them have a length smaller than 32 tokens'.format(100-len(length[length>32])/32000))
print('{}% of them have a length smaller than 64 tokens'.format(100-len(length[length>64])/32000))
print('{}% of them have a length smaller than 128 tokens'.format(100-len(length[length>128])/32000))
print('So we just use 64 as RoBERTa tokenizer max padding length')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
there are 3200000 entries in the dataframe
85.026% of them have a length smaller than 32 tokens
95.9338125% of them have a length smaller than 64 tokens
99.14103125% of them have a length smaller than 128 tokens
So we just use 64 as RoBERTa tokenizer max padding length


#### load tokenizer and encode the text data

In [32]:
tokenizer = RobertaTokenizer.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')
len(tokenizer)

50265

In [33]:
input_ids = []
attention_mask = []
for i in text:
    encoded_data = tokenizer.encode_plus(
    i,
    add_special_tokens=True,
    truncation=True,
    max_length=64,
    padding='max_length',
    return_attention_mask= True,
    return_tensors='pt')
    input_ids.append(encoded_data['input_ids'])
    attention_mask.append(encoded_data['attention_mask'])
input_ids = torch.cat(input_ids,dim=0)
attention_mask = torch.cat(attention_mask,dim=0)
labels = torch.tensor(labels)

#### fix random seed and split train/validation/test dataset to dataloader

In [34]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [35]:
from torch.utils.data import DataLoader,SequentialSampler,RandomSampler,TensorDataset,random_split

dataset = TensorDataset(input_ids,attention_mask,labels)
train_size = int(0.98*len(dataset))
val_size = int((len(dataset) - train_size)/2)
test_size = val_size

train_dataset,val_dataset = random_split(dataset,[train_size,2*val_size])
val_dataset, test_dataset = random_split(val_dataset,[val_size,test_size])
print('Training Size - ',train_size)
print('Validation Size - ',val_size)
print('Test Size - ',test_size)

Training Size -  3136000
Validation Size -  32000
Test Size -  32000


In [36]:
train_dl = DataLoader(train_dataset,sampler = RandomSampler(train_dataset),
                     batch_size = 128)
val_dl = DataLoader(val_dataset,sampler = SequentialSampler(val_dataset),
                     batch_size = 128)
test_dl = DataLoader(test_dataset,sampler = SequentialSampler(test_dataset),
                     batch_size = 128)
len(train_dl),len(val_dl),len(test_dl)

(24500, 250, 250)

#### load model and put to cuda and setup optimizer and scheduler

In [37]:
model = RobertaForSequenceClassification.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [39]:
optimizer = AdamW(model.parameters(),lr = 3e-5,eps=1e-8,no_deprecation_warning=True)

In [40]:
from transformers import get_linear_schedule_with_warmup
epochs = 1
total_steps = len(train_dl)*epochs
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)

#### define evaluation functions

In [41]:
def accuracy(preds,labels):
    pred_flat = np.argmax(preds,axis=1).flatten()
    label_flat = labels.flatten()
    return np.sum(pred_flat==label_flat)/len(label_flat)

In [42]:
from sklearn.metrics import classification_report
def report(preds,labels):
  target_names = ['Bearish','Bullish']
  pred_flat = np.argmax(preds,axis=1).flatten()
  label_flat = labels.flatten()
  print(classification_report(label_flat,pred_flat,target_names=target_names,digits=4))

In [43]:
def evaluate(dataloader_test):
    model.eval()
    loss_val_total = 0
    predictions,true_vals = [],[]
    for batch in dataloader_test:
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids':batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        with torch.no_grad():
            outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    loss_val_avg = loss_val_total / len(dataloader_test)
    predictions = np.concatenate(predictions,axis=0)
    true_vals = np.concatenate(true_vals,axis=0)
    return loss_val_avg,predictions,true_vals

#### actual training loop

In [ ]:
from tqdm.notebook import tqdm
torch.cuda.empty_cache()
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(train_dl, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(train_dl)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(val_dl)
    val_acc = accuracy(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'Validation Accuracy: {val_acc}')
    report(predictions,true_vals)    

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/98000 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.2106375729977771
Validation loss: 0.16033244659332557
Validation Accuracy: 0.93434375
              precision    recall  f1-score   support

     Bearish     0.9144    0.9579    0.9357     15956
     Bullish     0.9561    0.9109    0.9329     16044

    accuracy                         0.9343     32000
   macro avg     0.9353    0.9344    0.9343     32000
weighted avg     0.9353    0.9343    0.9343     32000



#### holdout testset evaluation

In [44]:
test_loss,preds_test,true_test = evaluate(test_dl)
report(preds_test,true_test)

              precision    recall  f1-score   support

     Bearish     0.9136    0.9573    0.9349     16027
     Bullish     0.9550    0.9092    0.9315     15973

    accuracy                         0.9333     32000
   macro avg     0.9343    0.9332    0.9332     32000
weighted avg     0.9342    0.9333    0.9332     32000



#### save output model and tokenizer

In [ ]:
output_dir = 'sentimentEngine4/'
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('sentimentEngine4/tokenizer_config.json',
 'sentimentEngine4/special_tokens_map.json',
 'sentimentEngine4/vocab.json',
 'sentimentEngine4/merges.txt',
 'sentimentEngine4/added_tokens.json')

#### result for all epochs

In [ ]:
Epoch 1
Training loss: 0.34953850715657075
Validation loss: 0.2956085407484323
Accuracy: 0.86789375
            precision    recall  f1-score   support

     Bearish     0.8465    0.8876    0.8666     15967
     Bullish     0.8859    0.8452    0.8651     16034

    accuracy                         0.8679     32000
   macro avg     0.8688    0.8679    0.8679     32000
weighted avg     0.8688    0.8679    0.8679     32000

In [ ]:
Epoch 2
Training loss: 0.27165976355687875
Validation loss: 0.2235213685684697
Accuracy: 0.90213652
            precision    recall  f1-score   support

     Bearish     0.8840    0.9248    0.9039     16031
     Bullish     0.9221    0.8793    0.9002     15069

    accuracy                         0.9021     32000
   macro avg     0.9030    0.9021    0.9021     32000
weighted avg     0.9030    0.9021    0.9021     32000

In [ ]:
Epoch 3
Training loss: 0.23595350905638948
Validation loss: 0.1874651527861133
Validation Accuracy: 0.921
            precision    recall  f1-score   support

     Bearish     0.9029    0.9437    0.9228     16021
     Bullish     0.9409    0.8982    0.9191     15979

    accuracy                         0.9210     32000
   macro avg     0.9219    0.9210    0.9210     32000
weighted avg     0.9219    0.9210    0.9210     32000

In [ ]:
Epoch 4
Training loss: 0.2106375729977771
Validation loss: 0.16033244659332557
Validation Accuracy: 0.93434375
              precision    recall  f1-score   support

     Bearish     0.9144    0.9579    0.9357     15956
     Bullish     0.9561    0.9109    0.9329     16044

    accuracy                         0.9343     32000
   macro avg     0.9353    0.9344    0.9343     32000
weighted avg     0.9353    0.9343    0.9343     32000

### Sentiment engine inferencing

#### Inferencing functions and demo

In [ ]:
!pip install transformers
!pip install emoji

In [5]:
import transformers
import torch
import math
import pandas as pd
import numpy as np
from transformers import RobertaForSequenceClassification, RobertaTokenizer, BertForSequenceClassification, BertTokenizer, AutoModelForSequenceClassification, AutoTokenizer, AdamW
import random
import time

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [6]:
tokenizer_loaded = RobertaTokenizer.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')
model_loaded = RobertaForSequenceClassification.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/782 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Checksenti() to check a single sentence

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def Sentiment(sent,model=model_loaded,tokenizer=tokenizer_loaded):
  encoded_dict = tokenizer.encode_plus(
                      sent, 
                      add_special_tokens = True,
                      truncation=True,
                      max_length = 64,
                      padding='max_length',
                      return_attention_mask = True,
                      return_tensors = 'pt')
      
  input_id = torch.LongTensor(encoded_dict['input_ids']).to(device)
  attention_mask = torch.LongTensor(encoded_dict['attention_mask']).to(device)
  model = model.to(device)

  with torch.no_grad():
      outputs = model(input_id, token_type_ids=None, attention_mask=attention_mask)

  logits = outputs[0]
  index = logits.argmax()
  return index,logits

!pip install emoji
import emoji
import re

def process_text(texts):
  # lowercase
  # message = message.lower() # RoBERTa tokenizer is uncased
  # remove URLs
  texts = re.sub(r'https?://\S+', "", texts)
  texts = re.sub(r'www.\S+', "", texts)
  # remove '
  texts = texts.replace('&#39;', "'")
  # remove symbol names
  texts = re.sub(r'(\#)(\S+)', r'hashtag_\2', texts)
  texts = re.sub(r'(\$)([A-Za-z]+)', r'cashtag_\2', texts)
  # remove usernames
  texts = re.sub(r'(\@)(\S+)', r'mention_\2', texts)
  # demojize
  texts = emoji.demojize(texts, delimiters=("", " "))

  return texts.strip()

def checkSenti(sent,return_logits=False):
  labels = ['Bearish','Bullish']
  sent_processed = process_text(sent)
  index,logits = Sentiment(sent_processed)
  if return_logits:
    logit0 = math.exp(logits[0][0])
    logit1 = math.exp(logits[0][1])
    logits = [logit0/(logit0+logit1),logit1/(logit0+logit1)]
    return labels[index],logits
  return labels[index]


batch_checkSenti() use GPU to perform batch inferencing

In [14]:
from torch.utils.data import DataLoader,SequentialSampler,TensorDataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batch_checkSenti(texts,model=model_loaded,tokenizer=tokenizer_loaded,return_logits=False):
  start = time.time()
  labels = ['Bearish','Bullish']
  input_ids = []
  attention_masks = []
  for text in texts:
      i = process_text(text)
      encoded_data = tokenizer.encode_plus(
      i,
      add_special_tokens=True,
      truncation=True,
      max_length=64,
      padding='max_length',
      return_attention_mask= True,
      return_tensors='pt')
      input_ids.append(encoded_data['input_ids'])
      attention_masks.append(encoded_data['attention_mask'])
  input_ids = torch.cat(input_ids,dim=0)
  attention_masks = torch.cat(attention_masks,dim=0)
  model.to(device)
  print('It takes {}s to tokenize'.format(time.time()-start))
  checkpointtime = time.time()

  testset = TensorDataset(input_ids,attention_masks)
  test_dl = DataLoader(testset,sampler = SequentialSampler(testset),batch_size = 128)

  predictions = []
  for batch in test_dl:
      batch = tuple(b.to(device) for b in batch)
      inputs = {
          'input_ids':batch[0],
          'attention_mask': batch[1],
      }

      with torch.no_grad():
          outputs = model(**inputs)
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      predictions.append(logits)

  predictions = np.concatenate(predictions,axis=0)
  index = predictions.argmax(axis=1)
  print('It takes {}s to do predictions'.format(time.time()-checkpointtime))

  # if return_logits:
  #   return index,predictions
  return (index,predictions) if return_logits else index

Use checkSenti()

In [ ]:
samples = ['just buy','just sell it','entity rocket to the sky!','go down','even though it is going up, I still think it will go down in the near future']
for sample in samples:
  print(checkSenti(sample))

Bullish
Bearish
Bullish
Bearish
Bearish


Use batch_checkSenti()

In [ ]:
samples_num = 20000
testing = batch_checkSenti(df.body.iloc[:samples_num])
true_labels = df.entities.iloc[:samples_num].replace('Bullish',1).replace('Bearish',0)
# len(['same' for i in range(samples_num) if testing[i]==true_labels[i]])/samples_num
testing

It takes 13.196947813034058s to tokenize
It takes 144.93988299369812s to do predictions


array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(true_labels,testing))

              precision    recall  f1-score   support

           0       0.88      0.85      0.86     10000
           1       0.85      0.89      0.87     10000

    accuracy                           0.87     20000
   macro avg       0.87      0.87      0.87     20000
weighted avg       0.87      0.87      0.87     20000



#### test 20000 unseen samples and check performance

In [25]:
df_testset = pd.read_csv('testset20000.csv')

In [26]:
df_testset

,Unnamed: 0,body,entities,Stock
0,0,$FB 💚☘️💚☘️💚☘️215 at least today. Don’t be greed.,Bullish,FB
1,1,$AMZN Patience is the key.. $2800 is a hard s...,Bullish,AMZN
2,2,$AAPL $162 open,Bullish,AAPL
3,3,$HYMC 4600 watchers--yesterday at 5 we had 360...,Bullish,AMZN
4,4,$NVDA what has NVDA to do with Russis/ Ukraine?,Bullish,NVDA
...,...,...,...,...
19995,19995,$FB,Bearish,FB
19996,19996,$TSLA Looks like this afternoon was just a nas...,Bearish,TSLA
19997,19997,$FB WAITING,Bearish,FB
19998,19998,$TSLA Thursday alway is a shit day short it,Bearish,TSLA


In [23]:
true_labels = df_testset.entities.replace('Bullish',1).replace('Bearish',0)
predictions = batch_checkSenti(df_testset.body)

It takes 49.61083912849426s to tokenize
It takes 588.8205716609955s to do predictions


In [24]:
from sklearn.metrics import classification_report

print(classification_report(true_labels,predictions,digits=4))

              precision    recall  f1-score   support

           0     0.9206    0.9328    0.9266     80143
           1     0.9316    0.9193    0.9254     79857

    accuracy                         0.9260    160000
   macro avg     0.9261    0.9260    0.9260    160000
weighted avg     0.9261    0.9260    0.9260    160000



#### test tweeter sentiment dataset and check performance

In [15]:
df_tweeter = pd.read_csv('twitterData.csv',encoding='latin1',header=None)
df_tweeter.columns=['Sentiment', 'id', 'Date', 'Query', 'User', 'Tweet']
df_tweeter = df_tweeter.drop(columns=['id', 'Date', 'Query', 'User'], axis=1)

In [16]:
df_tweeter['Sentiment'] = df_tweeter.Sentiment.replace(4,1)

In [17]:
df_tweeter_samples = df_tweeter.sample(frac=0.1,random_state=42).reset_index(drop=True)
df_tweeter_samples

,Sentiment,Tweet
0,0,@chrishasboobs AHHH I HOPE YOUR OK!!!
1,0,"@misstoriblack cool , i have no tweet apps fo..."
2,0,@TiannaChaos i know just family drama. its la...
3,0,School email won't open and I have geography ...
4,0,upper airways problem
...,...,...
159995,1,@willjonz thank you Will
159996,0,@HeartlessHippie AAAAAAAAH what's wrong?!
159997,1,@samhouston Will see you there tomorrow at the...
159998,0,Forgot that Xbox Live would be down today.


In [18]:
true_labels = df_tweeter_samples.Sentiment
predictions = batch_checkSenti(df_tweeter_samples.Tweet)

It takes 59.26477122306824s to tokenize
It takes 570.0794243812561s to do predictions


In [19]:
from sklearn.metrics import classification_report

print(classification_report(true_labels,predictions,digits=4))

              precision    recall  f1-score   support

           0     0.6145    0.3369    0.4352     79812
           1     0.5447    0.7896    0.6447     80188

    accuracy                         0.5638    160000
   macro avg     0.5796    0.5633    0.5399    160000
weighted avg     0.5795    0.5638    0.5402    160000



#### Train model on tweeter sentiment dataset, and test on Stocktwits dataset
#### Not good, so we give up train sentiment engine on tweeter sentiment dataset from kaggle

**RoBERTa(fine tuned on stocktwits) on unseen testset**

**accuracy =  0.84975**

**precision =  0.8612000413095116**

**recall =  0.8339**

**f1 score =  0.8473301834070011**

RoBERTa(fine tuned on stocktwits)

accuracy =  0.8866

precision =  0.9020178905762429

recall =  0.867373474694939

f1 score =  0.8843565164185192

RoBERTa(fine tuned on 1.6m tweets)

accuracy =  0.6249

precision =  0.5970775687859474

recall =  0.7682

f1 score =  0.6719146330796816

BERT(fine tuned on 1.6m tweets)

accuracy =  0.6165

precision =  0.5983454330575722

recall =  0.7088

f1 score =  0.6489059782111141

FinBERT(fine tuned on 1.6m tweets)

accuracy =  0.6124

precision =  0.5935419440745673

recall =  0.7132

f1 score =  0.6478924418604652

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [ ]:
get_available_gpus()

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import pipeline
import pandas as pd

tokenizer_loaded = RobertaTokenizer.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')
model_loaded = RobertaForSequenceClassification.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')

nlp = pipeline("text-classification", model=model_loaded, tokenizer=tokenizer_loaded)

sentences = pd.Series(['just buy','just sell it','entity rocket to the sky!','go down','even though it is going up, I still think it will not keep this trend in the near future'])
sentences = list(sentences.apply(process_text))
results = nlp(sentences)
print(results) 

[{'label': 'LABEL_1', 'score': 0.9866015315055847}, {'label': 'LABEL_0', 'score': 0.9912450909614563}, {'label': 'LABEL_1', 'score': 0.9932643175125122}, {'label': 'LABEL_0', 'score': 0.9969961643218994}, {'label': 'LABEL_0', 'score': 0.9047526717185974}]


### RobertaForSequenceClassification Structure

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
model.train()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

### Test pretraining (test only)

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="roberta-base",
    tokenizer="roberta-base"
)
fill_mask("Send these <mask> back!")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

[{'score': 0.16661493480205536,
  'sequence': 'Send these pictures back!',
  'token': 3493,
  'token_str': ' pictures'},
 {'score': 0.10792786628007889,
  'sequence': 'Send these photos back!',
  'token': 2356,
  'token_str': ' photos'},
 {'score': 0.07670937478542328,
  'sequence': 'Send these emails back!',
  'token': 5575,
  'token_str': ' emails'},
 {'score': 0.04860762506723404,
  'sequence': 'Send these images back!',
  'token': 3156,
  'token_str': ' images'},
 {'score': 0.04841742664575577,
  'sequence': 'Send these letters back!',
  'token': 5430,
  'token_str': ' letters'}]

In [ ]:
!git clone https://github.com/cardiffnlp/tweeteval /content/tmp/tweeteval

from transformers import RobertaTokenizer, RobertaForMaskedLM

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

Cloning into '/content/tmp/tweeteval'...
remote: Enumerating objects: 367, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 367 (delta 21), reused 13 (delta 4), pack-reused 318
Receiving objects: 100% (367/367), 10.79 MiB | 17.41 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/tmp/tweeteval/datasets/hate/train_text.txt",
    block_size=512,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta-retrained",
    overwrite_output_dir=True,
    num_train_epochs=25,
    per_device_train_batch_size=48,
    save_steps=500,
    save_total_limit=2,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train()

In [ ]:
# trainer.save_model("./roberta-retrained")